In [18]:
import io
import os
import tempfile
from pathlib import Path

from rich import print as rprint

from entitysdk.client import Client
from entitysdk.common import ProjectContext
from entitysdk.models import (
    BrainLocation,
    BrainRegion,
    ReconstructionMorphology,
    Species,
    Strain,
)

### Initialize the entitycore client

In [19]:
entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730",
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6",
)
token = os.getenv("ACCESS_TOKEN", "XXX")
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)


# uncomment for staging
# from obi_auth import get_token
# token = get_token(environment="staging")
# Replace this with your vlab project url in staging
# project_context = ProjectContext.from_vlab_url("https://staging.openbraininstitute.org/app/virtual-lab/lab/594fd60d-7a38-436f-939d-500feaa13bba/project/ff89ca07-6613-4922-9ab0-2637221db8b5/home")
# client = Client(environment="staging", project_context=project_context)

## Search for entities

In [20]:
species = client.search_entity(entity_type=Species, query={"name": "Mus musculus"}, limit=10).one()

In [21]:
rprint(species)

Species(
    id=UUID('fbb190bf-593a-4a89-bd81-fcb6e4c5c133'),
    update_date=datetime.datetime(2025, 6, 11, 7, 12, 51, 883561, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 6, 11, 7, 12, 51, 883561, tzinfo=TzInfo(UTC)),
    name='Mus musculus',
    taxonomy_id='NCBITaxon:10090'
)

In [22]:
strain = client.search_entity(entity_type=Strain, query={"name": "Cux2-CreERT2"}).one()

IteratorResultError: Iterable is empty.

In [23]:
rprint(strain)

NameError: name 'strain' is not defined

In [24]:
brain_region = client.search_entity(entity_type=BrainRegion, query={"annotation_value": 68}).one()

In [25]:
rprint(brain_region)

BrainRegion(
    id=UUID('ef1772f4-100c-4e80-a132-3c2481b377bd'),
    update_date=datetime.datetime(2025, 6, 11, 7, 12, 37, 910887, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 6, 11, 7, 12, 37, 910887, tzinfo=TzInfo(UTC)),
    name='Frontal pole, layer 1',
    annotation_value=68,
    acronym='FRP1',
    parent_structure_id=UUID('ec1ca538-d956-4c07-aa9b-e4ddf0b0bed8'),
    hierarchy_id=UUID('f728b4fa-4248-4e3a-8a5d-2f346baa9455'),
    color_hex_triplet='268F45'
)

### Create a morphology object

In [26]:
brain_location = BrainLocation(
    x=4101.52490234375,
    y=1173.8499755859375,
    z=4744.60009765625,
)
morphology = ReconstructionMorphology(
    name="my-morph",
    description="A morphology",
    species=species,
    strain=strain,
    brain_region=brain_region,
    location=brain_location,
    legacy_id=None,
)

NameError: name 'strain' is not defined

In [27]:
rprint(morphology)

NameError: name 'morphology' is not defined

## Register morphology

In [28]:
registered = client.register_entity(entity=morphology)

NameError: name 'morphology' is not defined

## Upload assets

In [29]:
with tempfile.TemporaryDirectory() as tdir:
    file1 = Path(tdir, "morph.h5")
    file1.write_text("h5")

    file2 = Path(tdir, "morph.swc")
    file2.write_text("swc")

    # use a filepath to register first asset
    asset1 = client.upload_file(
        entity_id=registered.id,
        entity_type=ReconstructionMorphology,
        file_path=file1,
        file_content_type="application/h5",
    )
    rprint(asset1)

    # use an in-memory buffer to upload second asset
    buffer = io.BytesIO(b"morph bytes buffer")

    asset2 = client.upload_content(
        entity_id=registered.id,
        entity_type=ReconstructionMorphology,
        file_content=buffer,
        file_name="buffer.h5",
        file_content_type="application/swc",
    )
    rprint(asset2)

NameError: name 'registered' is not defined

## Retrieve it

In [12]:
# with assets included (default)
fetched = client.get_entity(entity_id=registered.id, entity_type=ReconstructionMorphology)

NameError: name 'registered' is not defined

In [ ]:
rprint(fetched)

In [15]:
# without assets
fetched_wout_assets = client.get_entity(
    entity_id=registered.id, entity_type=ReconstructionMorphology, with_assets=False
)

NameError: name 'registered' is not defined

In [ ]:
rprint(fetched_wout_assets)

## Download asset

In [ ]:
downloaded_asset = client.download_assets(
    fetched.assets,
    selection={"content_type": "application/swc"},
    output_path="./my-file.h5",
).one()

content = client.download_content(
    entity_id=fetched.id, entity_type=type(fetched), asset_id=downloaded_asset.id
)

print(content)
print(Path("my-file.h5").read_text())

## Delete asset

In [16]:
for asset in fetched.assets:
    if asset.content_type == "application/swc":
        print("Deleting asset ", asset.id)
        deleted_asset = client.delete_asset(
            entity_id=fetched.id,
            entity_type=type(registered),
            asset_id=asset.id,
        )
        break

rprint(deleted_asset)

fetched = client.get_entity(
    entity_id=registered.id, entity_type=ReconstructionMorphology, token=token
)
rprint(fetched.assets)

NameError: name 'fetched' is not defined

## Update asset

In [17]:
# update h5 asset with another file
with tempfile.TemporaryDirectory() as tdir:
    file1 = Path(tdir, "updated.h5")
    file1.write_text("updated h5")
    file1.touch()

    for asset in fetched.assets:
        if asset.content_type == "application/h5":
            updated_asset = client.update_asset_file(
                entity_id=fetched.id,
                entity_type=type(registered),
                asset_id=asset.id,
                file_name=asset1.path,
                file_path=file1,
                file_content_type=asset.content_type,
            )
            break


# synchronize morphology with new server data
fetched = client.get_entity(
    entity_id=registered.id, entity_type=ReconstructionMorphology, token=token
)
rprint(fetched.assets)


# download updated file and check it was successfully updated
for asset in fetched.assets:
    if asset.content_type == "application/h5":
        client.download_file(
            entity_id=fetched.id,
            entity_type=type(fetched),
            asset_id=updated_asset.id,
            output_path="./my-file.h5",
            token=token,
        )
        content = client.download_content(
            entity_id=fetched.id, entity_type=type(fetched), asset_id=asset.id, token=token
        )
        break
print(content)

NameError: name 'fetched' is not defined

## Search it

In [ ]:
hits = client.search_entity(
    entity_type=ReconstructionMorphology,
    query={"name__ilike": "my-morph", "page": 1, "page_size": 2},
    limit=None,
).all()

print("Number of results: ", len(hits))

In [ ]:
rprint(hits[0])